<img src="images/GAlogo.png" style="float: left; margin: 15px; height: 100px">

# CAPSTONE PROJECT
## US TORNADOES AND HOW TO PREDICT THEIR MAGNITUDE
### Notebook for importing the mulitple csv files and copying them into local PostgreSQL database

In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup

%matplotlib inline

In [2]:
# To avoid display of warnings in Jupyter Notebook:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Link to all the csv files:
link = "https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/"
response = requests.get(link)
html = response.text

In [4]:
# Making a soup out of the html and creating the list of file names:
soup = BeautifulSoup(html)
elements = soup.findAll("a",{"class":""})
elements

[<a href="?C=N;O=D">Name</a>,
 <a href="?C=M;O=A">Last modified</a>,
 <a href="?C=S;O=A">Size</a>,
 <a href="?C=D;O=A">Description</a>,
 <a href="/pub/data/swdi/stormevents/">Parent Directory</a>,
 <a href="Storm-Data-Bulk-csv-Format.pdf">Storm-Data-Bulk-csv-Format.pdf</a>,
 <a href="Storm-Data-Export-Format.pdf">Storm-Data-Export-Format.pdf</a>,
 <a href="StormEvents_details-ftp_v1.0_d1950_c20210803.csv.gz">StormEvents_details-ftp_v1.0_d1950_c20210803.csv.gz</a>,
 <a href="StormEvents_details-ftp_v1.0_d1951_c20210803.csv.gz">StormEvents_details-ftp_v1.0_d1951_c20210803.csv.gz</a>,
 <a href="StormEvents_details-ftp_v1.0_d1952_c20210803.csv.gz">StormEvents_details-ftp_v1.0_d1952_c20210803.csv.gz</a>,
 <a href="StormEvents_details-ftp_v1.0_d1953_c20210803.csv.gz">StormEvents_details-ftp_v1.0_d1953_c20210803.csv.gz</a>,
 <a href="StormEvents_details-ftp_v1.0_d1954_c20210803.csv.gz">StormEvents_details-ftp_v1.0_d1954_c20210803.csv.gz</a>,
 <a href="StormEvents_details-ftp_v1.0_d1955_c20210

In [5]:
StormEventDetails_allyears = []

# The first 6 elements and the last 2 have to be discarded:
for element in elements[6:-2]:
    
    # Selecting only the storm events tables (one for each year)
    if element.attrs['href'].startswith('StormEvents_details'):
        
        filename = element.attrs['href']
        StormEventDetails_url = link + filename
        
        # Creating an iterator in order to load the file in chunks of 1000 elements:
        iter_csv = pd.read_csv(StormEventDetails_url, compression='gzip', iterator=True,
                               chunksize=1000)
        
        # Concatenating the different chunks into a single dataframe, 
        # selecting the tornado events only:
        StormEventDetails_allyears.append(
            pd.concat([chunk for chunk in iter_csv]))

# Concatenating all the dataframes from the different years:
StormEventDetails = pd.concat(StormEventDetails_allyears)


        
print(StormEventDetails.shape)
StormEventDetails.head(2)

(1811210, 51)


,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,195004,28,1445,195004,28,1445,NaN,10096222,OKLAHOMA,40.0,...,0.0,NaN,NaN,35.12,-99.2,35.17,-99.2,NaN,NaN,PUB
1,195004,29,1530,195004,29,1530,NaN,10120412,TEXAS,48.0,...,0.0,NaN,NaN,31.90,-98.6,31.73,-98.6,NaN,NaN,PUB


In [10]:
StormEventDetails.to_csv('StormEventDetails.csv', index=False)

In [6]:
eventTypeCount = StormEventDetails['EVENT_TYPE'].value_counts()

In [7]:
eventTypeCount.head(50)

EVENT_TYPE
Thunderstorm Wind           502986
Hail                        390036
Flash Flood                  97078
High Wind                    85790
Winter Storm                 84574
Tornado                      75522
Winter Weather               74428
Heavy Snow                   70384
Drought                      69851
Flood                        63689
Marine Thunderstorm Wind     36223
Heavy Rain                   29205
Heat                         25426
Strong Wind                  25247
Lightning                    17442
Cold/Wind Chill              16157
Blizzard                     15899
Dense Fog                    15689
Extreme Cold/Wind Chill      15664
Frost/Freeze                 14086
Ice Storm                    12142
Excessive Heat               11871
High Surf                    10387
Funnel Cloud                  9284
Wildfire                      8360
Tropical Storm                6366
Waterspout                    5683
Coastal Flood                 3762
Lake-Effe

In [11]:
# Creating DataFrame for 'Thunderstorm Wind'
StormEventDetails_ThunderstormWind = StormEventDetails[StormEventDetails['EVENT_TYPE'] == 'Thunderstorm Wind']

# Saving the DataFrame as a CSV file
StormEventDetails_ThunderstormWind.to_csv('StormEventDetails_ThunderstormWind.csv', index=False)

# Creating DataFrame for 'Hail'
StormEventDetails_Hail = StormEventDetails[StormEventDetails['EVENT_TYPE'] == 'Hail']

# Saving the DataFrame as a CSV file
StormEventDetails_Hail.to_csv('StormEventDetails_Hail.csv', index=False)

In [65]:
pd.set_option('display.max_columns', None)
df_Hail = StormEventDetails_Hail
df_Hail.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,STATE,STATE_FIPS,YEAR,MONTH_NAME,EVENT_TYPE,CZ_TYPE,CZ_FIPS,CZ_NAME,WFO,BEGIN_DATE_TIME,CZ_TIMEZONE,END_DATE_TIME,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,MAGNITUDE,BEGIN_RANGE,BEGIN_AZIMUTH,BEGIN_LOCATION,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,195509,22,1610,195509,22,1610,MISSOURI,29.0,1955,September,Hail,C,37,CASS,NaN,22-SEP-55 16:10:00,CST,22-SEP-55 16:10:00,0,0,0,0,0,0,NaN,1.50,0.0,NaN,NaN,0.0,NaN,NaN,38.70,-94.08,NaN,NaN,NaN,NaN,PUB
3,195507,19,1330,195507,19,1330,ALABAMA,1.0,1955,July,Hail,C,3,BALDWIN,NaN,19-JUL-55 13:30:00,CST,19-JUL-55 13:30:00,0,0,0,0,0,0,NaN,0.75,0.0,NaN,NaN,0.0,NaN,NaN,30.50,-87.90,NaN,NaN,NaN,NaN,PUB
10,195510,28,1645,195510,28,1645,ALABAMA,1.0,1955,October,Hail,C,117,SHELBY,NaN,28-OCT-55 16:45:00,CST,28-OCT-55 16:45:00,0,0,0,0,0,0,NaN,2.00,0.0,NaN,NaN,0.0,NaN,NaN,33.28,-86.78,NaN,NaN,NaN,NaN,PUB
14,195506,12,1700,195506,12,1700,COLORADO,8.0,1955,June,Hail,C,67,LA PLATA,NaN,12-JUN-55 17:00:00,CST,12-JUN-55 17:00:00,0,0,0,0,0,0,NaN,1.00,0.0,NaN,NaN,0.0,NaN,NaN,37.08,-107.80,NaN,NaN,NaN,NaN,PUB
17,195505,7,1845,195505,7,1845,KANSAS,20.0,1955,May,Hail,C,21,CHEROKEE,NaN,07-MAY-55 18:45:00,CST,07-MAY-55 18:45:00,0,0,0,0,0,0,NaN,1.75,0.0,NaN,NaN,0.0,NaN,NaN,37.08,-94.68,NaN,NaN,NaN,NaN,PUB


In [67]:
# Get the shape of the DataFrame
print(f"Number of rows: {df_Hail.shape[0]}")
print(f"Number of columns: {df_Hail.shape[1]}\n")

# Get the data types of the columns
print("Data types:")
print(df_Hail.dtypes)

# Get the number of missing values in each column
print("\nNumber of missing values in each column:")
print(df_Hail.isnull().sum())

Number of rows: 390036
Number of columns: 39

Data types:
BEGIN_YEARMONTH        int64
BEGIN_DAY              int64
BEGIN_TIME             int64
END_YEARMONTH          int64
END_DAY                int64
END_TIME               int64
STATE                 object
STATE_FIPS           float64
YEAR                   int64
MONTH_NAME            object
EVENT_TYPE            object
CZ_TYPE               object
CZ_FIPS                int64
CZ_NAME               object
WFO                   object
BEGIN_DATE_TIME       object
CZ_TIMEZONE           object
END_DATE_TIME         object
INJURIES_DIRECT        int64
INJURIES_INDIRECT      int64
DEATHS_DIRECT          int64
DEATHS_INDIRECT        int64
DAMAGE_PROPERTY       object
DAMAGE_CROPS          object
SOURCE                object
MAGNITUDE            float64
BEGIN_RANGE          float64
BEGIN_AZIMUTH         object
BEGIN_LOCATION        object
END_RANGE            float64
END_AZIMUTH           object
END_LOCATION          object
BEGIN_LAT     

In [23]:
# List of columns to drop
columns_to_drop = ['MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE', 'TOR_LENGTH', 'TOR_WIDTH',
                   'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE', 'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 
                   'EPISODE_ID', 'EVENT_ID']

df_Hail.drop(columns_to_drop, axis=1, inplace=True)

In [25]:
print(df_Hail.isnull().sum())

BEGIN_YEARMONTH           0
BEGIN_DAY                 0
BEGIN_TIME                0
END_YEARMONTH             0
END_DAY                   0
END_TIME                  0
STATE                     0
STATE_FIPS                0
YEAR                      0
MONTH_NAME                0
EVENT_TYPE                0
CZ_TYPE                   0
CZ_FIPS                   0
CZ_NAME                 516
WFO                   37866
BEGIN_DATE_TIME           0
CZ_TIMEZONE               0
END_DATE_TIME             0
INJURIES_DIRECT           0
INJURIES_INDIRECT         0
DEATHS_DIRECT             0
DEATHS_INDIRECT           0
DAMAGE_PROPERTY      154799
DAMAGE_CROPS         157619
SOURCE               107731
MAGNITUDE               101
BEGIN_RANGE           73281
BEGIN_AZIMUTH        147156
BEGIN_LOCATION        62926
END_RANGE             73424
END_AZIMUTH          154294
END_LOCATION          80515
BEGIN_LAT             20190
BEGIN_LON             20192
END_LAT               81996
END_LON             

In [37]:
df_Hail[df_Hail['DEATHS_DIRECT'] != 0].shape[0]

11

In [38]:
df_Hail[df_Hail['DEATHS_INDIRECT'] != 0].shape[0]

10

In [39]:
df_Hail[df_Hail['INJURIES_DIRECT'] != 0].shape[0]

326

In [40]:
df_Hail[df_Hail['INJURIES_INDIRECT'] != 0].shape[0]

18

In [48]:
# Handle missing values in 'MAGNITUDE'
df_Hail['MAGNITUDE'].fillna(df_Hail['MAGNITUDE'].median(), inplace=True)

In [64]:
pd.set_option('display.max_rows', 100)
df_Hail[df_Hail['MAGNITUDE'] == 0]

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,STATE,STATE_FIPS,YEAR,MONTH_NAME,EVENT_TYPE,CZ_TYPE,CZ_FIPS,CZ_NAME,WFO,BEGIN_DATE_TIME,CZ_TIMEZONE,END_DATE_TIME,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,MAGNITUDE,BEGIN_RANGE,BEGIN_AZIMUTH,BEGIN_LOCATION,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
45,195504,22,2215,195504,22,2215,OKLAHOMA,40.0,1955,April,Hail,C,17,CANADIAN,NaN,22-APR-55 22:15:00,CST,22-APR-55 22:15:00,0,0,0,0,0,0,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,35.50000,-97.68000,NaN,NaN,NaN,NaN,PUB
62,195508,15,1437,195508,15,1437,MONTANA,30.0,1955,August,Hail,C,67,PARK,NaN,15-AUG-55 14:37:00,CST,15-AUG-55 14:37:00,0,0,0,0,0,0,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,45.70000,-110.58000,NaN,NaN,NaN,NaN,PUB
93,195506,15,1825,195506,15,1825,OKLAHOMA,40.0,1955,June,Hail,C,45,ELLIS,NaN,15-JUN-55 18:25:00,CST,15-JUN-55 18:25:00,0,0,0,0,0,0,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,36.08000,-99.78000,NaN,NaN,NaN,NaN,PUB
190,195506,19,1850,195506,19,1850,NEW MEXICO,35.0,1955,June,Hail,C,9,CURRY,NaN,19-JUN-55 18:50:00,CST,19-JUN-55 18:50:00,0,0,0,0,0,0,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,34.38000,-103.18000,NaN,NaN,NaN,NaN,PUB
246,195506,25,100,195506,25,100,MISSOURI,29.0,1955,June,Hail,C,63,DE KALB,NaN,25-JUN-55 01:00:00,CST,25-JUN-55 01:00:00,0,0,0,0,0,0,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,39.88000,-94.58000,NaN,NaN,NaN,NaN,PUB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25286,200505,29,1623,200505,29,1628,NEW YORK,36.0,2005,May,Hail,C,33,FRANKLIN,BTV,29-MAY-05 16:23:00,EST,29-MAY-05 16:28:00,0,0,0,0,1K,NaN,TRAINED SPOTTER,0.0,NaN,NaN,ST REGIS FALLS,NaN,NaN,ST REGIS FALLS,44.66667,-74.53333,44.66667,-74.53333,A cold upper level trough across the northeast...,NaN,PDS
25287,200505,29,1620,200505,29,1625,NEW YORK,36.0,2005,May,Hail,C,33,FRANKLIN,BTV,29-MAY-05 16:20:00,EST,29-MAY-05 16:25:00,0,0,0,0,1K,NaN,TRAINED SPOTTER,0.0,NaN,NaN,PAUL SMITHS,NaN,NaN,PAUL SMITHS,44.43333,-74.25000,44.43333,-74.25000,A cold upper level trough across the northeast...,NaN,PDS
25288,200505,30,1545,200505,30,1600,VERMONT,50.0,2005,May,Hail,C,5,CALEDONIA,BTV,30-MAY-05 15:45:00,EST,30-MAY-05 16:00:00,0,0,0,0,1K,NaN,TRAINED SPOTTER,0.0,NaN,NaN,EAST BURKE,NaN,NaN,EAST BURKE,44.60000,-71.95000,44.60000,-71.95000,A cold upper level trough across the northeast...,NaN,PDS
52285,200505,9,1240,200505,9,1240,CALIFORNIA,6.0,2005,May,Hail,C,89,SHASTA,STO,09-MAY-05 12:40:00,PST,09-MAY-05 12:40:00,0,0,0,0,10K,NaN,LAW ENFORCEMENT,0.0,NaN,NaN,ANDERSON,NaN,NaN,ANDERSON,40.45000,-122.30000,40.45000,-122.30000,Hail accumulation up to one inch on I-5 result...,NaN,PDS


In [8]:
StormEventDetails.columns

Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE'],
      dtype='object')

In [9]:
# Doing the same for the other two tables (locations and fatalities):
StormEventFatalities_allyears = []
StormEventLocations_allyears = []

for element in elements[6:-2]:
    if element.attrs['href'].startswith('StormEvents_fatalities'):
        filename = element.attrs['href']
        StormEventFatalities_url = link+filename
        iter_csv = pd.read_csv(StormEventFatalities_url, compression='gzip', iterator=True,
                               chunksize=1000)
        StormEventFatalities_allyears.append(pd.concat(iter_csv))

    elif element.attrs['href'].startswith('StormEvents_locations'):
        filename = element.attrs['href']
        StormEventLocation_url = link+filename
        iter_csv = pd.read_csv(StormEventLocation_url, compression='gzip', iterator=True,
                               chunksize=1000)
        StormEventLocations_allyears.append(pd.concat(iter_csv))
        
StormEventFatalities = pd.concat(StormEventFatalities_allyears)
StormEventLocations = pd.concat(StormEventLocations_allyears)

print(StormEventFatalities.shape, StormEventLocations.shape)
StormEventFatalities.head(2)

(21518, 11) (1569705, 11)


,FAT_YEARMONTH,FAT_DAY,FAT_TIME,FATALITY_ID,EVENT_ID,FATALITY_TYPE,FATALITY_DATE,FATALITY_AGE,FATALITY_SEX,FATALITY_LOCATION,EVENT_YEARMONTH
0,195001,13,525,1005198,9981922,D,01/13/1950 05:25:00,NaN,NaN,NaN,195001.0
1,195002,12,1200,1005199,10049525,D,02/12/1950 12:00:00,NaN,NaN,NaN,195002.0


In [18]:
StormEventLocations.head(2)

,YEARMONTH,EPISODE_ID,EVENT_ID,LOCATION_INDEX,RANGE,AZIMUTH,LOCATION,LATITUDE,LONGITUDE,LAT2,LON2
0,197206,990000001,990000001,1,NaN,NaN,LABELLE,26.77,-81.48,2677.0,-8148.0
1,197206,990000001,990000001,2,NaN,NaN,LABELLE,26.78,-81.48,2678.0,-8148.0


In [22]:
# Creating engine connection to my local "storms" database, using sqlalchemy:
engine_local = create_engine('postgresql://localhost:5432/storms')

# Copying the first dataframe to local PostgreSQL:
StormEventDetails.to_sql('tornadoes_1950_mid2017', engine_local, index=False,
                         if_exists='replace')

In [23]:
# Copying the second dataframe
StormEventFatalities.to_sql('fatalities_1950_mid2017', engine_local, index=False,
                            if_exists='replace')

In [24]:
# Copying the third dataframe
StormEventLocations.to_sql('locations_1950_mid2017', engine_local, index=False,
                           if_exists='replace')

With the three tables now saved locally, we have a backup.<br>
Following is part 3 of the capstone, where the tables will be read directly from my local PostgreSQL.